In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import sys
sys.path.insert(1, '../src/')

from image_tools import *
from text_tools import *
from data_prepare import *
import keras
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.models import load_model

from keras.layers import Embedding, LSTM
import matplotlib.pyplot as plt
from keras import Sequential

from keras import backend as K

In [2]:
data_path = "../../ndsc_data/"
path_image = '../../ndsc_data/training_img/training_img/'

In [3]:
d_train = pd.read_csv(os.path.join(data_path, "new_training_set.csv"),
                      usecols=["title_1", "image_1", "title_2", "image_2", "Label"])
d_test = pd.read_csv(os.path.join(data_path, "new_test_sample.csv"))

In [4]:
d_train.shape

(10181, 5)

In [5]:
d_train.head()

,title_1,image_1,title_2,image_2,Label
0,Johnson’s ® Top to Toe Hair & Body Bath 500ml,fdff8b9b8229da091dd7d070aae05f81.jpg,Johnson's cottontouch top to toe hair & body b...,41e191742760932598c7bd201e5dad47.jpg,0
1,Sandal Humble,906cc44f0be72d4e767669b5b63e3a17.jpg,Sandal Humble Glass - Glanzton,7a556b836bfdd08ea592216440524a34.jpg,0
2,PROMO LIKUID LIKUIT LIQUIT BABY POD LIQUID SAL...,475c26635de18b9f93032400732ff336.jpg,Voporizer Liquit - Likuit - Likuid - Liquid Pr...,ace93bec689f3f1565800c500a8341fa.jpg,0
3,6 Pasang / Set Anting Tusuk Bentuk Lingkaran A...,e630997f6217555d6026547ad1c15f0b.jpg,Subei 6 Pasang / Set Anting Tusuk Boho Bohemia...,31abbc176b09f5bd1728cfc3ecbbfb9c.jpg,0
4,ROREC NATURAL SKIN CARE MASK ROREC SHEET MASK ...,a27d11700a7902febd039dc3a96f10f2.jpg,Rorec 86 Natural Skin Care Shert Mask All Variant,813ad9dd638c10f1765db9dde20c9e42.jpg,1


## Prepare Dataset

In [6]:
d_train2 = d_train.sample(300)
d_test2 = d_test.sample(20)

In [7]:
d_train2.title_2.isnull().sum()

0

In [8]:
d_train2.title_1 = d_train2.title_1.fillna('')
d_train2.title_2 = d_train2.title_2.fillna('')

d_train2['title_1_pre'] = d_train2.title_1.apply(text_cleansing)
d_train2['title_2_pre'] = d_train2.title_2.apply(text_cleansing)

d_test2['title_1_pre'] = d_test2.title_1.apply(text_cleansing)
d_test2['title_2_pre'] = d_test2.title_2.apply(text_cleansing)

In [9]:
d_train, d_test, word2idx, idx2word = data_text_prep(d_train2, d_test2)

In [10]:
VOCAB_SIZE = len(word2idx)
VOCAB_SIZE

1683

In [11]:
dataset_train = ShopeeDataset(d_train, d_test, word2idx, idx2word)
dataset_val = ShopeeDataset(d_train, d_test, word2idx, idx2word, stage = 'val')

In [12]:
len_train = dataset_train.dataset['train'][1]
t1_enc_tr, t2_enc_tr, i1_tr, i2_tr, label_tr = [],[],[],[],[]
for i in range(len_train):
    t1_encode, t2_encode, i1_scaled, i2_scaled, label0 = dataset_train.__getitem__(i)
    t1_enc_tr.append(t1_encode)
    t2_enc_tr.append(t2_encode)
    i1_tr.append(i1_scaled)
    i2_tr.append(i2_scaled)
    label_tr.append(label0)
    
t1_enc_tr = np.stack(t1_enc_tr)
t2_enc_tr = np.stack(t2_enc_tr)
i1_tr = np.stack(i1_tr)/255
i2_tr = np.stack(i2_tr)/255
label_tr = np.array(label_tr)


len_val = dataset_train.dataset['val'][1]
t1_enc_val, t2_enc_val, i1_val, i2_val, label_val = [],[],[],[],[]
for i in range(len_val):
    t1_encode, t2_encode, i1_scaled, i2_scaled, label0 = dataset_val.__getitem__(i)
    t1_enc_val.append(t1_encode)
    t2_enc_val.append(t2_encode)
    i1_val.append(i1_scaled)
    i2_val.append(i2_scaled)
    label_val.append(label0)
    
t1_enc_val = np.stack(t1_enc_val)
t2_enc_val = np.stack(t2_enc_val)
i1_val = np.stack(i1_val)/255
i2_val = np.stack(i2_val)/255    
label_val = np.array(label_val)

In [13]:
i1_tr.shape

(225, 224, 224, 3)

In [14]:
# for k in i1_tr:
#     if k.shape != (224, 224, 3):
#         print(k.shape)
#         X= k

In [15]:
label_tr = label_tr.astype(float)
label_val = label_val.astype(float)

## Model

In [16]:
def contrastive_loss(y_true, y_pred):
    margin = 0.5
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [17]:
def euclidean_distance(vect):
    x, y = vect
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

In [18]:
def image_net() :
    IMG_DIM = 224
    # input_img = keras.Input(shape=(IMG_DIM,IMG_DIM, 3))
    mod1 = Sequential()
    mod1.add(MobileNetV2(include_top=False,input_shape=(IMG_DIM,IMG_DIM,3)))
    mod1.add(layers.Conv2D(256, (3, 3), activation='relu'))
    mod1.add(layers.Reshape((25,256,1)))
    return mod1

def image_net() :
    IMG_DIM = 224
    # input_img = keras.Input(shape=(IMG_DIM,IMG_DIM, 3))
    mod1 = Sequential()
    mod1.add(MobileNetV2(include_top=False,input_shape=(IMG_DIM,IMG_DIM,3)))
    mod1.add(layers.Conv2D(256, (3, 3), activation='relu'))
    mod1.add(layers.Reshape((25,256,1)))
    return mod1

def text_net() :
    # sentence_indices = keras.Input(25, dtype = 'int32')
    mod1 = Sequential()
    mod1.add(Embedding(VOCAB_SIZE, 512))
    mod1.add(LSTM(256,return_sequences = True))
    mod1.add(layers.Reshape((25,256,1)))
    return mod1

def connect_net() :
    ##concate_input = keras.Input(shape=(25,512,1))
    mod1 = Sequential()
    mod1.add(layers.Conv2D(1, (3, 11), activation='relu'))
    mod1.add(layers.MaxPool2D((2, 2)))
    mod1.add(layers.Conv2D(1, (3, 11), activation='relu'))
    mod1.add(layers.MaxPool2D((2, 2)))
    mod1.add(layers.Conv2D(1, (3, 11), activation='relu'))
    mod1.add(layers.MaxPool2D((2, 2)))
    mod1.add(layers.Reshape((55,)))
    return mod1



In [19]:
input_dim_img = (224,224,3)
input_dim_txt = (25)
eucl_dist_output_shape = 1

im_net = image_net()
tx_net = text_net()
cn_net = connect_net()

img_a = keras.Input(shape=input_dim_img)
img_b = keras.Input(shape=input_dim_img)

txt_a = keras.Input(shape=input_dim_txt)
txt_b = keras.Input(shape=input_dim_txt)

feat_img_a = im_net(img_a)
feat_img_b = im_net(img_b)

feat_txt_a = tx_net(txt_a)
feat_txt_b = tx_net(txt_b)

concat_A = layers.Concatenate(axis=2)([feat_img_a, feat_txt_a])
concat_B = layers.Concatenate(axis=2)([feat_img_b, feat_txt_b])

vec_a = cn_net(concat_A)
vec_b = cn_net(concat_B)

simmilarity_dist = layers.Lambda(euclidean_distance, output_shape = eucl_dist_output_shape)([vec_a ,vec_b])

prediction = layers.Dense(1,activation='softmax')(simmilarity_dist)
model_shopi = keras.Model(inputs = [txt_a, txt_b, img_a, img_b], outputs = prediction)

In [20]:
model_shopi.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 25)]         0                                            
_______________________________________________________________________________________

## Train

In [21]:
filepath = '../../ndsc_model/model_shopi/model_{epoch:02d}_{val_loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)

In [22]:
opt = keras.optimizers.Adam(learning_rate=0.001)
# opt = keras.optimizers.RMSprop(learning_rate=0.0005)
model_shopi.compile(optimizer=opt, loss=contrastive_loss, metrics=['accuracy'])

In [ ]:
enc_train = model_shopi.fit([t1_enc_tr, t2_enc_tr, i1_tr, i2_tr],label_tr,
                epochs=50,
                batch_size=15,
                shuffle=True,
                validation_data=([t1_enc_val, t2_enc_val, i1_val, i2_val], label_val),
                   callbacks=[checkpoint])

Epoch 1/50
15/15 [==============================] - ETA: 0s - loss: 0.6044 - accuracy: 0.6044
Epoch 00001: val_loss improved from inf to 0.50667, saving model to ../../ndsc_model/model_shopi/model_01_0.5067.hdf5
15/15 [==============================] - 58s 4s/step - loss: 0.6044 - accuracy: 0.6044 - val_loss: 0.5067 - val_accuracy: 0.5067
Epoch 2/50
 2/15 [===>..........................] - ETA: 23s - loss: 0.7667 - accuracy: 0.7667